### Transmission:
We encode the 80 bytes textfile bit(byte) by bit(byte). <br>
Each bit(byte) is encoded in a 1(128)-bits codeword. <br>
We form an intermediary 80(512) bits vector X_i (for byte i) that consists of 80(4) repetitions of the codeword. <br>
Once we've done this for all bytes in the textfile, we concatenate all intermediary vectors X_i to obtain a 640 * 80=51200(512 * 80=40960) bits vector X.

##### Byte to codeword mapping:
   Let k be the bit to encode. If k=1 then codeword = +1, if k=0 then codeword = -1.

   (Let k be the value represented by the byte. <br>
   Let l = k mod 128. <br>
   The corresponding codeword is the l-th column of matrix M, with a + sign if l<= 128, - sign if l>128.)

### Reception:
We receive a 51200(40960) bits vector Y = X + Z, with Z_i iid N(0,11). <br>
Divide Y in 51200/80(40960/80) intermediary subvectors Y_i of length 640(512) bits. <br>
...


In [1]:
import numpy as np
import math

In [2]:
# GLOBAL VARIABLES

N = 51200 #40960
CODEWORD_SIZE = 128 #if m = 2⁸, m = 2n => n = 2⁷
X_I_SIZE = 640 #512
REPETITION = 5 #4

# with CODEWORD_SIZE = 2^i
i = 7

## Transmission

In [3]:
def compute_M(j):
    if j== 0:
        return np.array([1])
    M_1 = np.array([[1,1],[1,-1]])
    temp = M_1
    for l in range (j-1):
        up = np.concatenate((temp,temp), axis=1) 
        bottom = np.concatenate((temp,-temp), axis=1)
        temp = np.concatenate((up,bottom), axis=0)
    return temp

In [4]:
def columnk_M(M,k):
    if M.shape == (1,):
        return M[0]
    if k>=0 and k<np.size(M, 1):
        return M[:,k-1]
    else:
        print("bad index for column: "+str(k))


In [5]:
def encode_mk(M,k,sign):
    return (sign*columnk_M(M,k))

In [6]:
def transmitter3(filename):
    
    f=open(filename, 'rb')
    data = f.read()
    f.close()
    X = np.array([])
    sign = 1
    print ("len data = ")
    print (len(data))
    print("last byte = ")
    print(data[len(data)-1])
    for j in range (len(data)-1):
        #print (data[j])
        
        # get the byte
        if (data[j]<=CODEWORD_SIZE):
            sign = 1
        else :
            sign = -1
        
        l = np.mod(data[j], CODEWORD_SIZE)
            
        # create intermediary REPETITION*CODEWORD_SIZE bits X_i
        X_i = np.array([])
        for r in range (REPETITION):
            V = encode_mk(compute_M(i),l,sign)
            X_i = np.concatenate((X_i,V), axis=0)
        
        # append intermediary X_i to N-bits vector X
        X = np.concatenate((X,X_i), axis=0)
        
    print ("X size = ")
    print (X.size)

    return np.savetxt("signalX.txt", X, fmt="%s")
    

In [33]:
transmitter3("pdc_input_alphabet.txt")

len data = 
81
last byte = 
10
X size = 
51200


In [34]:
import os
print(os.stat("pdc_input_alphabet.txt"))

os.stat_result(st_mode=33204, st_ino=20983818, st_dev=66307, st_nlink=1, st_uid=1001, st_gid=1001, st_size=81, st_atime=1591097537, st_mtime=1589307782, st_ctime=1591094098)


In [40]:
f=open("signalX.txt", 'rb')
data1 = f.read()
f.close()
print ("len data = ")
print (len(data1))
print(chr(data1[len(data)-1]))

len data = 
230400
1


In [12]:
#Remove last blank byte from textfile for linux system:
# truncate -s -1 file

In [41]:
f=open("signalX.txt", 'rb')
data2 = f.read()
f.close()
print ("len data = ")
print (len(data2))

len data = 
230399


In [42]:
tx_p_signal = np.loadtxt("signalX.txt")
len(tx_p_signal)

51200

    python3 client.py --input_file=signalX.txt --output_file=output.txt --srv_hostname=iscsrv72.epfl.ch --srv_port=80

## Reception

In [22]:
# Y_i is a X_I_SIZE-bits subvector of N-bits Y.
# j should correspond to the global var i

def compute_U(M,Y_i,j): 
    if j==0:
        return Y_i
    if j==1:
        return np.dot(M,Y_i)
    U_bis = compute_M(j-1)
    
    U_t = np.dot(U_bis,Y_i[0:int(CODEWORD_SIZE/2)])
    U_b = np.dot(U_bis,Y_i[int(CODEWORD_SIZE/2):])
    return np.concatenate((U_t+U_b,U_t-U_b),axis=0)   

In [23]:
def decode_U(U): # returns the value of the decoded text byte
    U_bis=abs(U)
    k=np.argmax(U_bis)
    if U[int(k)]<0:
        return -(k+1)
    else :
        return k+1

In [24]:
def receiver(filename):
    
    # open filename to get corresponding binary vector (N-bits) Y
    Y = np.genfromtxt(filename)
    Y=list(Y)
    output = open("decoded_text.txt", 'w')
    Y_0 = np.array([])
    Y_1 = np.array([])
    Y_2 = np.array([])
    Y_3 = np.array([])
    
    # for all subvector (X_I_SIZE) Y_i in Y, we decode the corresponding byte
    for j in range(0,X_I_SIZE*80, X_I_SIZE):
        Y_i = Y[j:j+X_I_SIZE]
        #print (Y_i)
        Y_0 = Y_i[0:128]
        Y_1 = Y_i[128:256]
        Y_2 = Y_i[256:384]
        Y_3 = Y_i[384:512]
        curr_byte = decode_U(compute_U(compute_M(i), Y_0, i)+compute_U(compute_M(i), Y_1, i)+compute_U(compute_M(i), Y_2, i)+compute_U(compute_M(i), Y_3, i))
        #print(curr_byte)
        print(chr(curr_byte))
        
        # each time we get a byte, we write it in a new output file
        output.write(chr(curr_byte))
        
    output.close()
        

In [25]:
receiver("output.txt")

T
h
i
s
 
t
e
x
t
 
t
e
a
c
h
e
s
 
b
a
s
i
c
s
 
d
i
g
i
t
a
l
 
c
o
m
m
u
n
i
c
a
t
i
o
n
 
v
i
a
 
a
 
t
o
p
-
d
o
w
n
-
r
e
s
e
r
v
e
d
 
a
p
p
r
o
a
c
h
.


## All together:

In [43]:
M=compute_M(2)
print(M)


[[ 1  1  1  1]
 [ 1 -1  1 -1]
 [ 1  1 -1 -1]
 [ 1 -1 -1  1]]


In [45]:
columnk_M(M,2)

array([ 1, -1,  1, -1])

In [47]:
M2=compute_M(0)
print(M2.shape)

(1,)


In [48]:
print(encode_mk(M,3,-1))

[-1 -1  1  1]


In [50]:
M1=compute_M(i)
U1=compute_U(M1,encode_mk(M1,1,-1),i)

In [52]:
decode_U(U1)

-1

In [57]:
f = open("pdc_input_alphabet.txt", 'rb')
Y = f.read(2)
f.close()
Y

b'az'